In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from functools import reduce
import matplotlib as plt

In [2]:
#import csv of oscar winners
oscar_movie_db=pd.read_csv("./DataSources/Oscars1934-2015.csv")
pd.to_numeric(oscar_movie_db.Year, errors='coerce')

#Ensure that all the "Names" in the oscar_movie_db are formated correctly (no spaces at end of name)
for i,row in oscar_movie_db.iterrows():    
    j = oscar_movie_db['Name'].iloc[i]
    if j[-1] != " ":
        pass
    else:
        j=j[:-1]
        oscar_movie_db['Name'].iloc[i]=j

#fix Moulin Rouge 1952 and Moulin Rouge 2001 (remake of movie) so that these movies are treated distinct
for i,film in oscar_movie_db.iterrows():
    if oscar_movie_db['Name'][i]=='Moulin Rouge' and oscar_movie_db['Year'][i]==1952:
        oscar_movie_db['Name'][i]='Moulin Rouge 1952'
    if oscar_movie_db['Name'][i]=='Moulin Rouge' and oscar_movie_db['Year'][i]==2001:
        oscar_movie_db['Name'][i]='Moulin Rouge 2001'
    if oscar_movie_db['Film'][i]=='Moulin Rouge' and oscar_movie_db['Year'][i]==1952:
        oscar_movie_db['Film'][i]='Moulin Rouge 1952'
    if oscar_movie_db['Film'][i]=='Moulin Rouge' and oscar_movie_db['Year'][i]==2001:
        oscar_movie_db['Film'][i]='Moulin Rouge 2001'      

#create a movie list of best picture movie winners and nominees list
df_movies = oscar_movie_db[((oscar_movie_db['Award']== 'Best Picture')|(oscar_movie_db['Award']== 'Best Motion Picture'))]
df_movies=df_movies.reset_index(drop=True)
movies= list(df_movies['Name'])
        
#create a best picture winner movie list
df_winners = oscar_movie_db[((oscar_movie_db['Award']== 'Best Picture')|(oscar_movie_db['Award']== 'Best Motion Picture'))&(oscar_movie_db['Winner'] == 1)]
df_winners=df_winners.reset_index(drop=True)
winners= list(df_winners['Name'])

#create a best picture nominee list
df_losers = oscar_movie_db[((oscar_movie_db['Award']== 'Best Picture')|(oscar_movie_db['Award']== 'Best Motion Picture'))&(oscar_movie_db['Winner'] == 0)]
df_losers=df_losers.reset_index(drop=True)
losers= list(df_losers['Name'])

#Best Picture dataframe (winners and losers)
df_bestPicture=oscar_movie_db[((oscar_movie_db['Award']== 'Best Picture')|(oscar_movie_db['Award']== 'Best Motion Picture'))]
df_bestPicture=df_bestPicture.reset_index(drop=True)
df_bestPicture=df_bestPicture[["Name", "Winner", "Film", "Year"]].rename(columns={"Winner": "Best Picture", "Film": "Prod Company", "Name":"Title"})

#Best Actor dataframe
df_bestActor=oscar_movie_db[((oscar_movie_db['Award']== 'Actor')|(oscar_movie_db['Award']== 'Actor in a Leading Role'))]
df_bestActor=df_bestActor.reset_index(drop=True)
df_bestActor=df_bestActor[["Film", "Winner"]].rename(columns={"Winner": "Best Actor", "Film": "Title"})
#correct for mulitple cast nominations
df_bestActor=df_bestActor.groupby('Title', as_index=False).agg({"Best Actor":np.sum})

#Best Actress dataframe
df_bestActress=oscar_movie_db[((oscar_movie_db['Award']== 'Actress')|(oscar_movie_db['Award']== 'Actress in a Leading Role'))]
df_bestActress=df_bestActress.reset_index(drop=True)
df_bestActress=df_bestActress[["Film", "Winner"]].rename(columns={"Winner": "Best Actress", "Film": "Title"})
#correct for mulitple cast nominations
df_bestActress=df_bestActress.groupby('Title', as_index=False).agg({"Best Actress":np.sum})

#Best Supporting Actor dataframe
df_bestSuppActor=oscar_movie_db[(oscar_movie_db['Award']== 'Actor in a Supporting Role')]
df_bestSuppActor=df_bestSuppActor.reset_index(drop=True)
df_bestSuppActor=df_bestSuppActor[["Film", "Winner"]].rename(columns={"Winner": "Best Supporting Actor", "Film": "Title"})
#correct for mulitple cast nominations
df_bestSuppActor=df_bestSuppActor.groupby('Title', as_index=False).agg({"Best Supporting Actor":np.sum})

#Best Supporting Actress dataframe
df_bestSuppActress=oscar_movie_db[(oscar_movie_db['Award']== 'Actress in a Supporting Role')]
df_bestSuppActress=df_bestSuppActress.reset_index(drop=True)
df_bestSuppActress=df_bestSuppActress[["Film", "Winner"]].rename(columns={"Winner": "Best Supporting Actress", "Film": "Title"})
#correct for mulitple cast nominations
df_bestSuppActress=df_bestSuppActress.groupby('Title', as_index=False).agg({"Best Supporting Actress":np.sum})

#Best Directors Database dataframe
df_bestDirectors=oscar_movie_db[(oscar_movie_db['Award']== 'Directing')]
df_bestDirectors=df_bestDirectors.reset_index(drop=True)
df_bestDirectors=df_bestDirectors[["Name", "Winner"]].rename(columns={"Winner": "Best Director", "Name": "Title"})
#correct for mulitple cast nominations
df_bestDirectors=df_bestDirectors.groupby('Title', as_index=False).agg({"Best Director":np.sum})


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value i

In [3]:
#OMBD Calls - done on movies list
theMoviesDF_OMBD = pd.read_csv('./DataSources/theMoviesDF_OMBD.csv')
theMoviesDF_OMBD.head()

#correct for remake of Moulin Rouge (able to distinguish between 2001 and 1952 versions)
for i,row in theMoviesDF_OMBD.iterrows():    
    j = theMoviesDF_OMBD['Title'][i]
    if j=='Moulin Rouge' and theMoviesDF_OMBD['Year'][i]==1952:
        j='Moulin Rouge 1952'
        theMoviesDF_OMBD['Title'][i]=j
    if j=='Moulin Rouge!' and theMoviesDF_OMBD['Year'][i]==2001:
        j='Moulin Rouge 2001'
        theMoviesDF_OMBD['Title'][i]=j

theMoviesDF_OMBD

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


,Actors,Awards,Box Office,Country,Director,Genre,Rated,Release Date,Title,Writer(s),Year,imdbRating
0,"Fred MacMurray, Barbara Stanwyck, Edward G. Ro...",Nominated for 7 Oscars. Another 2 wins & 2 nom...,NaN,USA,Billy Wilder,"Crime, Drama, Film-Noir, Mystery, Thriller",PASSED,06 Jul 1944,Double Indemnity,"Billy Wilder (screenplay), Raymond Chandler (s...",1944,8.3
1,"Charles Boyer, Ingrid Bergman, Joseph Cotten, ...",Won 2 Oscars. Another 3 wins & 7 nominations.,NaN,USA,George Cukor,"Crime, Drama, Mystery, Thriller",NOT RATED,30 Oct 1944,Gaslight,"John Van Druten (screenplay), Walter Reisch (s...",1944,7.8
2,"Bing Crosby, Barry Fitzgerald, Frank McHugh, J...",Won 7 Oscars. Another 11 wins & 4 nominations.,NaN,USA,Leo McCarey,"Comedy, Drama, Music, Musical",PASSED,02 Oct 1944,Going My Way,"Frank Butler (screenplay), Frank Cavett (scree...",1944,7.2
3,"Claudette Colbert, Jennifer Jones, Joseph Cott...",Won 1 Oscar. Another 8 nominations.,NaN,USA,"John Cromwell, Edward F. Cline, Tay Garnett, D...","Drama, Romance, War",APPROVED,20 Jul 1944,Since You Went Away,"Margaret Buell Wilder (book), Margaret Buell W...",1944,7.6
4,"Woody Harrelson, Sandy Oian-Thomas, Shaun Brow...",1 nomination.,"$652,997",USA,Craig Johnson,"Comedy, Drama",R,24 Mar 2017,Wilson,"Daniel Clowes (graphic novel), Daniel Clowes (...",2017,5.8
5,"Frank Sinatra, Kathryn Grayson, Gene Kelly, Jo...",Won 1 Oscar. Another 4 nominations.,NaN,USA,George Sidney,"Comedy, Fantasy, Musical",PASSED,01 Aug 1945,Anchors Aweigh,"Isobel Lennart (screen play), Natalie Marcin (...",1945,7.2
6,"Bing Crosby, Ingrid Bergman, Henry Travers, Wi...",Won 1 Oscar. Another 3 wins & 7 nominations.,NaN,USA,Leo McCarey,Drama,UNRATED,21 Feb 1946,The Bells of St. Mary's,"Dudley Nichols (screenplay), Leo McCarey (story)",1945,7.4
7,"Ray Milland, Jane Wyman, Phillip Terry, Howard...",Won 4 Oscars. Another 12 wins & 3 nominations.,NaN,USA,Billy Wilder,"Drama, Film-Noir",NOT RATED,01 Jan 1946,The Lost Weekend,"Charles R. Jackson (from the novel by), Charle...",1945,8.0
8,"Joan Crawford, Jack Carson, Zachary Scott, Eve...",Won 1 Oscar. Another 2 wins & 6 nominations.,NaN,USA,Michael Curtiz,"Crime, Drama, Film-Noir, Mystery, Romance",PASSED,20 Oct 1945,Mildred Pierce,"Ranald MacDougall (screenplay), James M. Cain ...",1945,8.0
9,"Ingrid Bergman, Gregory Peck, Michael Chekhov,...",Won 1 Oscar. Another 1 win & 6 nominations.,NaN,USA,Alfred Hitchcock,"Film-Noir, Mystery, Romance, Thriller",NOT RATED,28 Dec 1945,Spellbound,"Ben Hecht (screen play), John Palmer (suggeste...",1945,7.6


In [4]:
#TMDB Calls
theMoviesDF_TMDB=pd.read_csv('./DataSources/all_tmbd_movie_info4.csv').rename(columns={"Name":"Title"})

#remove space at end of title name and correct for remake of Moulin Rouge (able to distinguish between 2001 and 1952 versions)
for i,row in theMoviesDF_TMDB.iterrows():    
    j = theMoviesDF_TMDB['Title'][i]
    if j[-1] != " ":
        pass
    else:
        j=j[:-1]
        theMoviesDF_TMDB['Title'][i]=j
    if j=='Moulin Rouge' and theMoviesDF_TMDB['Year'][i]==1952:
        j='Moulin Rouge 1952'
        theMoviesDF_TMDB['Title'][i]=j
    if j=='Moulin Rouge' and theMoviesDF_TMDB['Year'][i]==2001:
        j='Moulin Rouge 2001'
        theMoviesDF_TMDB['Title'][i]=j

theMoviesDF_TMDB

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Year,Ceremony,Award,Winner,Title,Film,Revenue,Budget,Runtime,Vote_average_list,Vote_count,Popularity,Genre,Release_Date,MovieID
0,1944,17,Best Motion Picture,0,Double Indemnity,Paramount,2500000,927262,107,8.2,643,11.853,Crime,1944-04-24,996
1,1944,17,Best Motion Picture,0,Gaslight,Metro-Goldwyn-Mayer,0,2068000,114,7.4,118,6.868,Drama,1944-05-04,13528
2,1944,17,Best Motion Picture,1,Going My Way,Paramount,16300000,0,130,7.1,51,3.592,Drama,1944-05-15,17661
3,1944,17,Best Motion Picture,0,Since You Went Away,Selznick International Pictures,0,0,172,6.3,10,1.400,Drama,1944-07-20,42401
4,1944,17,Best Motion Picture,0,Wilson,20th Century-Fox,2000000,2995000,154,5.5,6,0.771,Romance,1944-08-01,84084
5,1945,18,Best Motion Picture,0,Anchors Aweigh,Metro-Goldwyn-Mayer,0,0,143,7.0,66,4.672,Comedy,1945-07-14,17889
6,1945,18,Best Motion Picture,0,The Bells of St. Mary's,Rainbow Productions,21300000,0,126,7.4,53,2.620,Drama,1945-12-27,32430
7,1945,18,Best Motion Picture,1,The Lost Weekend,Paramount,11000000,1250000,101,7.6,185,5.871,Drama,1945-11-29,28580
8,1945,18,Best Motion Picture,0,Mildred Pierce,Warner Brothers,5638000,1453000,111,7.7,127,6.637,Crime,1945-10-20,3309
9,1945,18,Best Motion Picture,0,Spellbound,Selznick International Pictures,7000000,1500000,111,7.4,279,8.192,Drama,1945-12-28,4174


In [5]:
#fix any title discrepancies between OMBD and TMDB dataframes by replacing incorrect OMBD names with TMDB names
#OMBD does not have data on the following movies: Precious: Based on the Novel 'Push' by Sapphire, The Postman (Il Postino), Cries and Whispers, Nashville 
bad_names_OMBD=[]
correct_names_TMDB=[]

for i in theMoviesDF_OMBD["Title"].values:
    if i in theMoviesDF_TMDB["Title"].values:
        pass
    else:
        bad_names_OMBD.append(i)


for i in theMoviesDF_TMDB["Title"].values:
    if i in theMoviesDF_OMBD["Title"].values:
        pass
    else:
        correct_names_TMDB.append(i)

bad_names_OMBD

['All About Eve',
 "The Librarian: Return to King Solomon's Mines",
 'Decision Before Dawn',
 "Right Here in River City: The Making of Meredith Willson's 'The Music Man'",
 'The Russians Are Coming, the Russians Are Coming: The Russians Are Coming to Hollywood',
 'The Godfather: Part II',
 "One Flew Over the Cuckoo's Nest",
 'Star Wars: Episode IV - A New Hope',
 'E.T. the Extra-Terrestrial',
 'Dances with Wolves',
 'The Godfather: Part III',
 'Secrets']

In [6]:
correct_names_TMDB

['All about Eve',
 "King Solomon's Mines",
 'Decision before Dawn',
 "Meredith Willson's The Music Man",
 'The Russians Are Coming The Russians Are Coming',
 'Cries and Whispers',
 'The Godfather Part II',
 'Nashville',
 "One Flew over the Cuckoo's Nest",
 'Star Wars',
 'E.T. The Extra-Terrestrial',
 'Dances With Wolves',
 'The Godfather, Part III',
 'The Postman (Il Postino)',
 'Secrets & Lies',
 "Precious: Based on the Novel 'Push' by Sapphire"]

In [7]:
#dictionary of OMBD Title name:TMBD Title name for common movies between "bad_names_OMBD" and "correct_names_TMDB"
name_correction={"All About Eve":"All about Eve",
                 "The Librarian: Return to King Solomon's Mines":"King Solomon's Mines",
                 "Decision Before Dawn":'Decision before Dawn',
                 "Right Here in River City: The Making of Meredith Willson's 'The Music Man'":"Meredith Willson's The Music Man",
                 "The Russians Are Coming, the Russians Are Coming: The Russians Are Coming to Hollywood":'The Russians Are Coming The Russians Are Coming',
                 "The Godfather: Part II":'The Godfather Part II',
                 "One Flew Over the Cuckoo's Nest":"One Flew over the Cuckoo's Nest",
                 "Star Wars: Episode IV - A New Hope":'Star Wars',
                 "E.T. the Extra-Terrestrial":'E.T. The Extra-Terrestrial',
                 "Dances with Wolves":'Dances With Wolves',
                 "The Godfather: Part III":'The Godfather, Part III',
                 "Secrets":'Secrets & Lies',
                 }

#replacing OMBD title name with TMDB title name based on name_correction dictionary 
for i in range(len(theMoviesDF_OMBD)): 
     for key, value in name_correction.items():
        if theMoviesDF_OMBD['Title'][i]==key:
            theMoviesDF_OMBD['Title'][i]=value

theMoviesDF_OMBD

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Actors,Awards,Box Office,Country,Director,Genre,Rated,Release Date,Title,Writer(s),Year,imdbRating
0,"Fred MacMurray, Barbara Stanwyck, Edward G. Ro...",Nominated for 7 Oscars. Another 2 wins & 2 nom...,NaN,USA,Billy Wilder,"Crime, Drama, Film-Noir, Mystery, Thriller",PASSED,06 Jul 1944,Double Indemnity,"Billy Wilder (screenplay), Raymond Chandler (s...",1944,8.3
1,"Charles Boyer, Ingrid Bergman, Joseph Cotten, ...",Won 2 Oscars. Another 3 wins & 7 nominations.,NaN,USA,George Cukor,"Crime, Drama, Mystery, Thriller",NOT RATED,30 Oct 1944,Gaslight,"John Van Druten (screenplay), Walter Reisch (s...",1944,7.8
2,"Bing Crosby, Barry Fitzgerald, Frank McHugh, J...",Won 7 Oscars. Another 11 wins & 4 nominations.,NaN,USA,Leo McCarey,"Comedy, Drama, Music, Musical",PASSED,02 Oct 1944,Going My Way,"Frank Butler (screenplay), Frank Cavett (scree...",1944,7.2
3,"Claudette Colbert, Jennifer Jones, Joseph Cott...",Won 1 Oscar. Another 8 nominations.,NaN,USA,"John Cromwell, Edward F. Cline, Tay Garnett, D...","Drama, Romance, War",APPROVED,20 Jul 1944,Since You Went Away,"Margaret Buell Wilder (book), Margaret Buell W...",1944,7.6
4,"Woody Harrelson, Sandy Oian-Thomas, Shaun Brow...",1 nomination.,"$652,997",USA,Craig Johnson,"Comedy, Drama",R,24 Mar 2017,Wilson,"Daniel Clowes (graphic novel), Daniel Clowes (...",2017,5.8
5,"Frank Sinatra, Kathryn Grayson, Gene Kelly, Jo...",Won 1 Oscar. Another 4 nominations.,NaN,USA,George Sidney,"Comedy, Fantasy, Musical",PASSED,01 Aug 1945,Anchors Aweigh,"Isobel Lennart (screen play), Natalie Marcin (...",1945,7.2
6,"Bing Crosby, Ingrid Bergman, Henry Travers, Wi...",Won 1 Oscar. Another 3 wins & 7 nominations.,NaN,USA,Leo McCarey,Drama,UNRATED,21 Feb 1946,The Bells of St. Mary's,"Dudley Nichols (screenplay), Leo McCarey (story)",1945,7.4
7,"Ray Milland, Jane Wyman, Phillip Terry, Howard...",Won 4 Oscars. Another 12 wins & 3 nominations.,NaN,USA,Billy Wilder,"Drama, Film-Noir",NOT RATED,01 Jan 1946,The Lost Weekend,"Charles R. Jackson (from the novel by), Charle...",1945,8.0
8,"Joan Crawford, Jack Carson, Zachary Scott, Eve...",Won 1 Oscar. Another 2 wins & 6 nominations.,NaN,USA,Michael Curtiz,"Crime, Drama, Film-Noir, Mystery, Romance",PASSED,20 Oct 1945,Mildred Pierce,"Ranald MacDougall (screenplay), James M. Cain ...",1945,8.0
9,"Ingrid Bergman, Gregory Peck, Michael Chekhov,...",Won 1 Oscar. Another 1 win & 6 nominations.,NaN,USA,Alfred Hitchcock,"Film-Noir, Mystery, Romance, Thriller",NOT RATED,28 Dec 1945,Spellbound,"Ben Hecht (screen play), John Palmer (suggeste...",1945,7.6


In [9]:
#pulling demographic info
# demographicsDB=pd.read_csv('./DataSources/OscarDemographics.csv', encoding='utf-8').rename(columns={"movie":"Title"})
# demographicsDB.head()

# EXTRACTING INFO FROM API CALLS

In [8]:
#DataFrame by Genre (OMBD)
genreMovies=theMoviesDF_OMBD[['Title', 'Genre']]

for i in range(len(genreMovies['Genre'])):
    x=genreMovies['Genre'][i].split(", ")
    genreMovies['Genre'][i]=x

s=genreMovies.apply(lambda x:pd.Series(x['Genre']), axis=1).stack().reset_index(level=1, drop=True)
s.name = 'Genre'
BP_genres=genreMovies.drop('Genre', axis=1).join(s)
BP_genres=pd.get_dummies(BP_genres, columns=['Genre'])
BP_genres=BP_genres.groupby('Title').sum()

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2961: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [9]:
#Dataframe by IMBD rating (OMBD)
BP_IMBD=theMoviesDF_OMBD[['Title','imdbRating']]

#Dataframe by Rating (OMBD)
BP_Rating_qual=theMoviesDF_OMBD[['Title','Rated']]
#consolidate certain ratings
BP_Rating_qual['Rated']=BP_Rating_qual['Rated'].replace({'GP':'PG',
                                                        'M/PG': 'PG',
                                                        'TV-MA': 'R',
                                                        'TV-PG': 'PG',
                                                        'PASSED':'APPROVED',
                                                        'NOT RATED': 'UNRATED'})
BP_Rating=pd.get_dummies(BP_Rating_qual, columns=['Rated'])

#Dataframe by Country (OMBD)
BP_Country_qual=theMoviesDF_OMBD[['Title','Country']]
BP_Country=pd.get_dummies(BP_Country_qual, columns=['Country'])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [10]:
#Dataframe by Revenue and Budget (TMDB)
BP_Revenue_Budget=theMoviesDF_TMDB[['Title','Revenue', 'Budget']]

#Dataframe by Popularity (TMDB)
BP_Popularity = theMoviesDF_TMDB[['Title','Popularity']]

#Dataframe by Run Time (TMDB)
BP_Runtime = theMoviesDF_TMDB[['Title','Runtime']]

In [13]:
#Dataframe for Release Month (TMDB)
BP_RelMonth_qual = theMoviesDF_TMDB[['Title','Release_Date']]
for i in range(len(BP_RelMonth_qual['Release_Date'])):
    x=BP_RelMonth_qual['Release_Date'][i].split("-")
    BP_RelMonth_qual['Release_Date'][i]=x[1]

BP_RelMonth=pd.get_dummies(BP_RelMonth_qual, columns=['Release_Date'])

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2961: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [14]:
#create a spreadsheet of all info (can have string as values)
theMoviesDF=pd.merge(df_bestPicture,BP_IMBD, on="Title", how='right')

dfs=[theMoviesDF, BP_Country_qual, BP_Country_qual, BP_Popularity, BP_Rating_qual, BP_RelMonth_qual, BP_Revenue_Budget, BP_Runtime, BP_genres, df_bestActor, df_bestActress, df_bestDirectors, df_bestSuppActor, df_bestSuppActress]
theMoviesDF_qual = reduce(lambda df1,df2: pd.merge(df1,df2,on=['Title'], how='left'), dfs)
theMoviesDF_qual.fillna(0, inplace=True)
theMoviesDF_qual.to_csv('theMovies_qual.csv', index=False)
theMoviesDF_qual.shape

(384, 41)

In [15]:
#create a spreadsheet of all info (cannot have string as values)
dfs=[theMoviesDF, BP_Popularity, BP_Rating, BP_RelMonth, BP_Revenue_Budget, BP_Runtime, BP_genres, df_bestActor, df_bestActress, df_bestDirectors, df_bestSuppActor, df_bestSuppActress]
theMoviesDF = reduce(lambda df1,df2: pd.merge(df1,df2,on=['Title'], how='left'), dfs)
theMoviesDF.fillna(0, inplace=True)
theMoviesDF.to_csv('theMovies.csv', index=False)
theMoviesDF.shape

(384, 55)